<a href="https://colab.research.google.com/github/prajwal-kotian/Machine-poet/blob/main/Beatles_txt_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q sklearn


In [2]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
import pandas as pd
import numpy as np

In [4]:
token = Tokenizer()
data = open('beatles.txt').read()

corpus = data.lower().split("\n")
print(corpus)

token.fit_on_texts(corpus)
total_words = len(token.word_index) +1
input_sequences = []
for line in corpus:
	token_list = token.texts_to_sequences([line])[0]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)
max_len_sent = max([len(x) for x in input_sequences])
input_seq =np.array( pad_sequences((input_sequences),padding = 'pre' ))


['yesterday, all my troubles seemed so far away', "now it looks as though they're here to stay", "oh, i believe in yesterday suddenly, i'm not half the man i used to be", "there's a shadow hanging over me.", "oh, yesterday came suddenly why she had to go i don't know she wouldn't say", 'i said something wrong, now i long for yesterday yesterday, love was such an easy game to play', 'now i need a place to hide away', "oh, i believe in yesterday why she had to go i don't know she wouldn't say", 'i said something wrong, now i long for yesterday yesterday, love was such an easy game to play', 'now i need a place to hide away', 'oh, i believe in yesterday', 'mm mm mm mm mm mm mm when i find myself in times of trouble, mother mary comes to me', 'speaking words of wisdom, let it be', 'and in my hour of darkness she is standing right in front of me', 'speaking words of wisdom, let it be', 'let it be, let it be, let it be, let it be', 'whisper words of wisdom, let it be and when the broken hear

In [5]:
xs, label = input_seq[:,:-1],input_seq[:,-1]
ys = tf.keras.utils.to_categorical(label , num_classes= total_words)

In [6]:
from tensorflow.keras import regularizers
model = tf.keras.Sequential()
model.add(Embedding(total_words, 100, input_length=max_len_sent-1))
model.add(Bidirectional(LSTM(150, return_sequences = True)))

model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words/2, activation='relu',kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(total_words, activation='softmax'))


In [7]:
model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy', metrics = ["accuracy"])
history = model.fit( xs ,ys , epochs = 100 , verbose = 1)

Epoch 1/100
392/392 [==============================] - 5s 13ms/step - loss: 6.1993 - accuracy: 0.0465
Epoch 2/100
392/392 [==============================] - 5s 13ms/step - loss: 5.6014 - accuracy: 0.0559
Epoch 3/100
392/392 [==============================] - 5s 13ms/step - loss: 5.4105 - accuracy: 0.0658
Epoch 4/100
392/392 [==============================] - 5s 14ms/step - loss: 5.2735 - accuracy: 0.0778
Epoch 5/100
392/392 [==============================] - 5s 13ms/step - loss: 5.1488 - accuracy: 0.0867
Epoch 6/100
392/392 [==============================] - 5s 13ms/step - loss: 5.0263 - accuracy: 0.1029
Epoch 7/100
392/392 [==============================] - 5s 13ms/step - loss: 4.9100 - accuracy: 0.1142
Epoch 8/100
392/392 [==============================] - 5s 13ms/step - loss: 4.7916 - accuracy: 0.1236
Epoch 9/100
392/392 [==============================] - 5s 13ms/step - loss: 4.6802 - accuracy: 0.1344
Epoch 10/100
392/392 [==============================] - 5s 13ms/step - loss: 4.578

In [8]:
seed_text = "My song"
next_words = 50
  
for _ in range(next_words):
	token_list = token.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences([token_list], maxlen=max_len_sent-1, padding='pre')
	predicted = model.predict_classes(token_list, verbose=0)
	output_word = ""
	for word, index in token.word_index.items():
		if index == predicted:
			output_word = word
			break
	seed_text += " " + output_word
print(seed_text)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
My song will fill the air we would shout and swim about a little help from my friends do you need anybody see the love there that's sleeping oh oh darling please please keep your hand and when i say i say yes you can rent a cottage who seems like years
